In [6]:
!pip install pdf2image

You should consider upgrading via the 'c:\python36\python.exe -m pip install --upgrade pip' command.


In [7]:
from pdf2image import convert_from_path, convert_from_bytes

In [8]:
from pdf2image.exceptions import (
    PDFInfoNotInstalledError,
    PDFPageCountError,
    PDFSyntaxError
)

In [5]:
poppler_path ="C:\poppler-21.03.0\Library\bin"

In [6]:
images = convert_from_path('012.GP 05-01-01_Latest.pdf')

KeyboardInterrupt: 

In [ ]:
for i in range(len(images)):
    page = images[i]
    page.save('output_{}.jpg'.format(i), 'JPEG')

In [9]:
from PIL import Image

In [5]:
!pip install pytesseract

You should consider upgrading via the 'c:\users\hp\anaconda3\python.exe -m pip install --upgrade pip' command.


In [10]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [7]:
#pytesseract.pytesseract.tesseract_cmd = r'C:\Users\HP\anaconda3\Scripts\pytesseract.exe'

pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

In [11]:
im=Image.open('output_10.jpg')

FileNotFoundError: [Errno 2] No such file or directory: 'output_10.jpg'

In [22]:
Image.fromqpixmap(pix)

AttributeError: 'Pixmap' object has no attribute 'hasAlphaChannel'

In [74]:
ocr =pytesseract.image_to_pdf_or_hocr(im)

In [22]:
with open('test.pdf', 'w+b') as f:
    f.write(pdf) # pdf type 

# Reading a full PDF

In [11]:
import fitz
from PIL import Image
doc2 = fitz.open()

import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


In [12]:
doc=fitz.open('Epson_04302021165418.pdf')

text=""
for page in doc:
    pix = page.getPixmap(matrix = fitz.Matrix(2,2),alpha=False)
    
    img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
    pix.writeImage("t.png")
    ocr=pytesseract.image_to_pdf_or_hocr(img)
    pdf = fitz.open("pdf", ocr)
    doc2.insertPDF(pdf)

doc2.save("fi.pdf")
    

In [56]:
"""
Demo script using Tesseract OCR.
Extract text of a page and interpret unrecognized characters using Tesseract.
MuPDF codes unrecognizable characters as 0xFFFD = 65533.
Extraction option is "dict", which delivers contiguous text pieces within one
line, that have the same font properties (color, fontsize, etc.). Together with
the language parameter, this helps Tesseract finding the correct character.
The basic approach is to only invoke OCR, if the span text contains
chr(65533). Because Tesseract's response ignores leading spaces and appends
line break characters, some adjustments are made.
--------------
This demo will OCR only text, that is known to be text. This means, it
does not look at parts of a page containing images or text encoded as drawings.
--------------
Dependencies:
Tesseract must be installed and invocable via Python's 'subprocess' module.
You also must install all the Tesseract language support you need to detect.
"""
import fitz
import subprocess
import time

# Tesseract invocation command (Windows version)
# Assume: language English. Detect more languages by add e.g. '+deu' for German.
# Assume: text represents one line (--psm 7)
# Note: Language mix spec increases duration by >40% - only use when needed!
tess = "tesseract stdin stdout --psm 7 -l eng"
mat = fitz.Matrix(4, 4)  # high resolution matrix
ocr_time = 0
pix_time = 0


def get_tessocr(page, bbox):
    """Return OCR-ed span text using Tesseract.
    Args:
        page: fitz.Page
        bbox: fitz.Rect or its tuple
    Returns:
        The OCR-ed text of the bbox.
    """
    global ocr_time, pix_time, tess, mat
    # Step 1: Make a high-resolution image of the bbox.
    t0 = time.perf_counter()
    pix = page.getPixmap(
        colorspace=fitz.csGRAY,  # we need no color
        matrix=mat,
        clip=bbox,
    )
    image = pix.getImageData("png")  # make a PNG image
    t1 = time.perf_counter()
    # Step 2: Invoke Tesseract to OCR the image. Text is stored in stdout.
    rc = subprocess.run(
        tess,  # the command
        input=image,  # the pixmap image
        stdout=subprocess.PIPE,  # find the text here
        shell=True,
    )

    # because we told Tesseract to interpret the image as one line, we now need
    # to strip off the line break characters from the tail.
    text = rc.stdout.decode()  # convert to string
    text = text[:-3]  # remove line end characters
    t2 = time.perf_counter()
    ocr_time += t2 - t1
    pix_time += t1 - t0
   
    return text


In [57]:


doc = fitz.open("6-12-0001.pdf")
ocr_count = 0
for page in doc:
    
    blocks = page.getText("dict", flags=0)["blocks"]
    for b in blocks:
        for l in b["lines"]:
            for s in l["spans"]:
                text = s["text"]
                if chr(65533) in text:  # invalid characters encountered!
                    # invoke OCR
                    ocr_count += 1
                    print("before: '%s'" % text)
                    text1 = text.lstrip()
                    sb = " " * (len(text) - len(text1))  # leading spaces
                    text1 = text.rstrip()
                    sa = " " * (len(text) - len(text1))  # trailing spaces
                    new_text = sb + get_tessocr(page, s["bbox"]) + sa
                    print(" after: '%s'" % new_text)

print("-------------------------")
print("OCR invocations: %i." % ocr_count)
print("Pixmap time: %g (avg %g) seconds." % (round(pix_time, 5), round(pix_time / ocr_count, 5)))
print("OCR time: %g (avg %g) seconds."% (round(ocr_time, 5), round(ocr_time / ocr_count, 5)))

before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'
 after: ''
before: '�'

In [49]:
blocks

[{'type': 0,
  'bbox': (75.30000305175781,
   76.01667785644531,
   523.1595458984375,
   86.75341796875),
  'lines': [{'wmode': 0,
    'dir': (1.0, 0.0),
    'bbox': (75.30000305175781,
     76.01667785644531,
     121.98338317871094,
     86.75341796875),
    'spans': [{'size': 7.7972002029418945,
      'flags': 16,
      'font': 'Arial-BoldMT',
      'color': 0,
      'text': 'GP 05-01-01 ',
      'origin': (75.30000305175781, 84.35968017578125),
      'bbox': (75.30000305175781,
       76.01667785644531,
       121.98338317871094,
       86.75341796875)}]},
   {'wmode': 0,
    'dir': (1.0, 0.0),
    'bbox': (265.017578125,
     76.01667785644531,
     331.609619140625,
     86.75341796875),
    'spans': [{'size': 7.7972002029418945,
      'flags': 16,
      'font': 'Arial-BoldMT',
      'color': 0,
      'text': 'Pressure Vessels ',
      'origin': (265.017578125, 84.35968017578125),
      'bbox': (265.017578125,
       76.01667785644531,
       331.609619140625,
       86.75341796

# Using PDF2Image and MuPDF

In [9]:
import tempfile
with tempfile.TemporaryDirectory() as path:
    images_from_path = convert_from_path('D495-MD-DWG-MD3-105D-405D01.pdf', output_folder=path)
    for i in images_from_path:
        ocr=pytesseract.image_to_pdf_or_hocr(i)
        pdf = fitz.open("pdf", ocr)
        doc2.insertPDF(pdf)

In [10]:
doc2.save("final.pdf")

In [10]:
from pdf2docx import Converter

pdf_file = 'in-42.3.pdf'
docx_file = 'sample.docx'

# convert pdf to docx
cv = Converter(pdf_file)
cv.convert(docx_file, start=0, end=None)
cv.close()

Parsing Page 1: 1/17...
Ignore page due to error: 'Page' object has no attribute 'getDrawings'
Parsing Page 2: 2/17...
Ignore page due to error: 'Page' object has no attribute 'getDrawings'
Parsing Page 3: 3/17...
Ignore page due to error: 'Page' object has no attribute 'getDrawings'
Parsing Page 4: 4/17...
Ignore page due to error: 'Page' object has no attribute 'getDrawings'
Parsing Page 5: 5/17...
Ignore page due to error: 'Page' object has no attribute 'getDrawings'
Parsing Page 6: 6/17...
Ignore page due to error: 'Page' object has no attribute 'getDrawings'
Parsing Page 7: 7/17...
Ignore page due to error: 'Page' object has no attribute 'getDrawings'
Parsing Page 8: 8/17...
Ignore page due to error: 'Page' object has no attribute 'getDrawings'
Parsing Page 9: 9/17...
Ignore page due to error: 'Page' object has no attribute 'getDrawings'
Parsing Page 10: 10/17...
Ignore page due to error: 'Page' object has no attribute 'getDrawings'
Parsing Page 11: 11/17...
Ignore page due to err

Exception: No parsed pages. Please parse page first.